In [1]:
import pandas as pd
import re
from tqdm.notebook import tqdm
import os
from datetime import datetime, timedelta, date
import numpy as np

### Load Project Codes csv
Download the Project Codes spreadsheet on google as a `.csv` file and save it to the parent directory of this notebook

In [97]:
proj_codes_df = pd.read_csv('../project_codes.csv')
proj_codes_df = proj_codes_df.drop(columns=['name'])
proj_codes_df.head()

,Project Code,Formal Name
0,ANO,Kiam Marcelo Junio
1,AP,The Algebra Project
2,AS,Afternoon Snatch
3,AV,Ambivert
4,B,BRUJOS


In [120]:
proj_codes_df.index=proj_codes_df['Project Code'] # sets the index as the project code

# Load the data we'll use for testing

In [121]:
def clean_df(df):
    df['project_id'] = df['Project ID'].fillna(df['Project Code'])
    df['project_id'] = df['project_id'].fillna(df['Unnamed: 0'])
    return df

In [122]:
def get_window_datetimes(file_name):
    file_name = file_name.replace("-", "_")
    re_string = r'\((.*?)\)' # regex string for finding window start and end dates

    match = re.findall(re_string, file_name)
    start = match[0].split("_")
    start_month = int(start[0])
    start_day = int(start[1])
    start_year = int(start[2])
    start_dt = datetime(month=start_month, day=start_day, year=start_year)
    
    end = match[1].split("_")
    end_month = int(end[0])
    end_day = int(end[1])
    end_year = int(end[2])
    end_dt = datetime(month=end_month, day=end_day, year=end_year)
    return start_dt, end_dt

In [123]:
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [124]:
# define locations of the files we want to use
data_dir = '../data/Spreadsheets_2019/'
time_window_dirs = os.listdir(data_dir) # << a list of the file names in the directory

In [125]:
vimeo_device_dfs = []
vimeo_region_dfs = []
vimeo_video_dfs = []
vimeo_date_dfs = []
vimeo_source_dfs = []

for window_dir in time_window_dirs:
    file_names = os.listdir(data_dir+window_dir)
    for file_name in file_names:
        start_dt, end_dt = get_window_datetimes(file_name)
        if 'Vimeo_Device' in file_name:
            vimeo_device_df = pd.read_csv(data_dir+window_dir+"/"+file_name)
            vimeo_device_df['start'] = start_dt
            vimeo_device_df['end'] = end_dt
            vimeo_device_dfs.append(vimeo_device_df)
        if 'Vimeo_Region' in file_name:
            vimeo_region_df = pd.read_csv(data_dir+window_dir+"/"+file_name)
            vimeo_region_df['start'] = start_dt
            vimeo_region_df['end'] = end_dt
            vimeo_region_dfs.append(vimeo_region_df)
        if 'Vimeo_Video' in file_name:
            video_df = pd.read_csv(data_dir+window_dir+"/"+file_name)
            video_df.columns = ['Project ID'] + video_df.columns[1:].to_list() # make the project id column name consistent
            video_df.plays = video_df.plays.replace('\r\n', np.nan)
            video_df = video_df[~video_df.isnull().all(axis=1)] # remove all completely null rows
            video_df['start'] = start_dt
            video_df['end'] = end_dt
            vimeo_video_dfs.append(video_df)
        if 'Vimeo_Date' in file_name:
            vimeo_date_df=pd.read_csv(data_dir+window_dir+"/"+file_name)
            vimeo_date_df['start'] = start_dt
            vimeo_date_df['end'] = end_dt
            vimeo_date_dfs.append(vimeo_date_df)
        if 'Vimeo_Source' in file_name:
            vimeo_source_df = pd.read_csv(data_dir+window_dir+"/"+file_name)
            vimeo_source_df['start'] = start_dt
            vimeo_source_df['end'] = end_dt
            vimeo_source_dfs.append(pd.read_csv(data_dir+window_dir+"/"+file_name))
            
vimeo_device_df = pd.concat(vimeo_device_dfs, axis=0, sort=False)
vimeo_region_df = pd.concat(vimeo_region_dfs, axis=0, sort=False)
vimeo_video_df = pd.concat(vimeo_video_dfs, axis=0, sort=False)
vimeo_date_df = pd.concat(vimeo_date_dfs, axis=0, sort=False)
vimeo_source_df = pd.concat(vimeo_source_dfs, axis=0, sort=False)

In [126]:
vimeo_video_df.head()

,Project ID,plays,downloads,loads,finishes,likes,comments,uri,name,duration,created_time,sizes,unique_loads,mean_percent,mean_seconds,sum_seconds,total_seconds,unique_viewers,start,end
0,BG,942,0.0,39932.0,109.0,1.0,0.0,/videos/189724238,Brown Girls -- Trailer,141.0,2016-11-01T01:03:06+00:00,Array,15724.0,60.0,41.0,39270.0,133678.0,540.0,2019-11-13,2019-12-10
1,BG,562,0.0,2399.0,110.0,1.0,1.0,/videos/203924325,Brown Girls -- Episode 1,410.0,2017-02-14T00:43:46+00:00,Array,1002.0,71.0,250.0,141000.0,230952.0,370.0,2019-11-13,2019-12-10
2,SFP,493,0.0,1150.0,21.0,1.0,0.0,/videos/143933380,Open TV Presents: Southern for Pussy,283.0,2015-10-28T22:38:55+00:00,Array,497.0,36.0,69.0,34114.0,139744.0,314.0,2019-11-13,2019-12-10
3,TRS,378,0.0,776.0,32.0,1.0,0.0,/videos/375475586,The Right Swipe - Episode 2,867.0,2019-11-25T18:26:55+00:00,Array,398.0,61.0,469.0,177449.0,328084.0,244.0,2019-11-13,2019-12-10
4,BG,243,0.0,587.0,70.0,1.0,0.0,/videos/204010747,Brown Girls -- Episode 2,874.0,2017-02-14T14:31:09+00:00,Array,276.0,75.0,599.0,145659.0,212609.0,171.0,2019-11-13,2019-12-10


# Simple Code Estimator
So far this seems to be working best even though it is the most simple

In [11]:
def estimate_proj_code(video_name):
    potential_codes = []
    for ind, row in proj_codes_df.iterrows():
        code, proj = row['Project Code'], row['Formal Name']
        if proj.lower() in video_name.lower():
            potential_codes.append(code)
    return potential_codes

In [12]:
estimated_codes = vimeo_video_df['name'].apply(estimate_proj_code)

In [129]:
x = len(estimated_codes) - np.sum(len([1 for i in estimated_codes if len(i) > 0]))
print("Estimated %s codes out of %s" % (x, len(vimeo_video_df)))

Estimated 1017 codes out of 3019


In [130]:
# TODO: Need to have a consistent way of double checking these against the project codes in the dataframe

# [Spacy PhraseMatcher](https://spacy.io/api/phrasematcher)
[Install Spacy](https://spacy.io/usage)

In [89]:
import spacy
nlp = spacy.load('en_core_web_sm')
from spacy.matcher import PhraseMatcher
phrase_matcher = PhraseMatcher(nlp.vocab)

In [102]:
# convert all the video titles to spacy objects
video_titles = [nlp(name.lower()) for name in vimeo_video_df['name']]

In [103]:
# get the matches for all video titles
match_ids = [phrase_matcher(vt) for vt in video_titles]

In [131]:
# convert the match string_ids to their code strings and cast to pandas series
codes = pd.Series([nlp.vocab.strings[match_id[0][0]] if len(match_id) > 0 else None for match_id in match_ids])

In [132]:
vimeo_video_df.iloc[2]['Project ID'] # NOTE: this is an example of a code that is bad, we'll need to figure out the best way to fix it

'SFP'

In [133]:
matches = []
for ind, row in vimeo_video_df.iterrows():
    if codes[ind] == row['Project ID']:
        matches.append(True)
    else:
        matches.append(False)
print("Successfully matched %s codes" % np.sum(matches))

Successfully matched 285 codes


### TODO: we need to do an error analysis of why this number of matches is so bad when the matching should be an easy problem